# Addtional Extraction for the Node List

1. Case Result, 
2. Author of the opinion text
3. Judge Panel
4. Opinion text


### Read text

In [1]:
import json
pdf2text = json.load(open("full_name_text.json", 'r'))

In [2]:
pdf2text.keys()

dict_keys(['file_name', 'raw_text'])

In [3]:
file_name = pdf2text['file_name']

In [4]:
raw_text = pdf2text['raw_text']

## 1. Case Result

In [5]:
import re

In [6]:
for layer in raw_text[10].split("\n"):
    if re.search(r'(AFFIRMED|APPEAL|VACATED|REVERSED|DISMISSED|AMENDED)', layer) != None:
        print(layer)

VACATED–IN–PART AND REMANDED.


In [7]:
decision = [None]*len(file_name)
for i in range(len(file_name)):
    for layer in raw_text[i].split("\n"):
        if re.search('^(AFFIRMED|APPEAL|VACATED|REVERSED|DISMISSED|AMENDED)', layer) != None:
            decision[i] = layer
    

In [8]:
de_no = []
for i in range(len(decision)):
    if decision[i] == None:
        de_no.append(i) 

In [9]:
len(de_no)

64

In [10]:
for i in range(len(file_name)):
    if decision[i] == None:
        for layer in raw_text[i].split("\n"):
            if re.search('(en banc.*denied\.|MOTION|petition.*denied\.|Remanded|Affirmed|reversed\.|Vacated and remanded|Petition granted)', layer) != None:
                decision[i] = layer

In [11]:
de_no = []
for i in range(len(decision)):
    if decision[i] == None:
        de_no.append(i) 

In [12]:
len(de_no)

8

## 2. Author of the Opinion Text

In [13]:
for layer in raw_text[9].split("\n"):
    if len(layer) <40 and \
    re.search(r'^(?!Before)(?!dissenting-in-part).*Judge.$', layer) != None:
        print(layer.split(",")[0])

LOURIE


In [14]:
author = [None]*len(file_name)
for i in range(len(file_name)):
    for layer in raw_text[i].split("\n"):
        if len(layer) <40 and \
        re.search(r'^(?!Before)(?!dissenting-in-part).*Judge.$', layer) != None:
            author[i] = layer.split(",")[0]

In [15]:
author_judge = author
for i in range(len(file_name)):
    if author[i]!=None and len(author[i].split("  "))>1:
        author_judge[i] = author[i].split("  ")[-1]
    
    

## 3. Judge Panel

In [16]:
judge_panel = [None]*len(file_name)
for i in range(len(file_name)):
        for layer in raw_text[i].split("\n"):
            if re.search(r'^Before(.*)Judge(.*)', layer) != None:
                    judge_panel[i] = layer

## 4. Opinion Text

### Identify the start point of the opinion text

In [17]:
op_start_layer = [None]*len(file_name)
for i in range(len(file_name)):
    for layer in raw_text[i].split("\n"):
        if len(layer) <40 and \
        re.search(r'^(?!Before)(?!dissenting-in-part).*Judge.$', layer) != None:
            op_start_layer[i] = layer

In [18]:
start_index = [None]*len(file_name)
for i in range(len(file_name)):
    if op_start_layer[i] != None:
        start_index[i] = (raw_text[i].split("\n").index(op_start_layer[i]))+1

### Identify the end point of the opinion text

In [19]:
op_end_layer = [None]*len(file_name)
for i in range(len(file_name)):
    for layer in raw_text[i].split("\n"):
        if re.search('^(AFFIRMED|APPEAL|VACATED|REVERSED|DISMISSED|AMENDED)', layer) != None:
            op_end_layer[i] = layer

In [20]:
end_index = [None]*len(file_name)
for i in range(len(file_name)):
    if op_end_layer[i] != None:
        end_index[i] = (raw_text[i].split("\n").index(op_end_layer[i]))

### Extract the opinion text based on the anchor 

In [21]:
opinion_text = [None]*len(file_name)

for i in range(len(file_name)):
    if start_index[i] != None and end_index[i]!= None:
        opinion_text[i]=''.join(raw_text[i].split("\n")[start_index[i]:end_index[i]])

In [22]:
op_no = []
for i in range(len(decision)):
    if opinion_text[i] == None:
        op_no.append(i) 

In [23]:
len(op_no)

83

## Store the Extractions as Meta Data

### Append to the node list `.csv`

In [1]:
import pandas as pd

In [26]:
meta = pd.read_csv("cite_node.csv", sep="\t")

In [27]:
meta.head()

,case_code,case_name,court_from,court_type
0,725 F.3d 1356,Apple Inc v International Trade Com'n,International Trade Commission,Federal Court
1,878 F.3d 1041,Regeneron Pharmaceuticals Inc v Merus NV,Southern District of New York,Federal District Court
2,857 F.3d 1323,Joseph Phelps Vineyards LLC v Fairmont Holding...,Trademark Trial and Appeal Board,Federal Court
3,809 F.3d 1295,Commonwealth Scientific and Indus Research Org...,Eastern District of Texas,Federal District Court
4,709 F.3d 1124,Radio Systems Corp v Lalor,Western District of Washington,Federal District Court


In [28]:
full_data = meta.assign(result = decision,
                  author = author_judge,
                  judge_panel = judge_panel)

In [18]:
full_data.to_csv('extraction854.csv',index=False, sep='\t')

### Store `author`, `result`, `opinion text`  to json

In [31]:
json.dump({'author': author_judge,
           'decision': decision,
           'opinion':opinion_text},open('decision_text.json','w'))

In [32]:
import json

In [33]:
op_data = json.load(open('decision_text.json','r'))

In [34]:
op_data.keys()

dict_keys(['author', 'decision', 'opinion'])

In [35]:
len(op_data['opinion'])

854